In [1]:
# lame training wheels
from sys import path
path += ['..']
#


import sopy as sp

/Users/quantum/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd
from pyscf import gto, scf
from sopy.tensorly import ext
from sopy.pyscf.ext import *

In [3]:

roll, pitch, yaw = 0*np.radians(np.pi/4), 0*np.radians(np.pi/2), 0*np.radians(-np.pi/4)

Run = pd.Series([ roll, pitch, yaw] , index = ['roll','pitch','yaw'])
Run

roll     0.0
pitch    0.0
yaw     -0.0
dtype: float64

In [ ]:
mol = gto.Mole()

if True:
    mol.build( atom = '''He 0 0 1;  ''', basis ='STO-3G',spin= 0)
    filename = 'He'
elif False:
    mol.build( atom = f'''H {rotation_matrix_from_euler(roll, pitch, yaw, [2,2,2])}; H {rotation_matrix_from_euler(roll, pitch, yaw, [2,-2,-2])}; H {rotation_matrix_from_euler(roll, pitch, yaw, [-2,2,-2])}; H {rotation_matrix_from_euler(roll, pitch, yaw, [-2,-2,2])} ''', basis ='STO-3G',spin= 0)
    filename = 'H4'    
##
elif True:
    coord =   """6.290730e-01,6.290730e-01,6.290730e-01
      6.290730e-01,-6.290730e-01 ,-6.290730e-01
     -6.290730e-01, 6.290730e-01 ,-6.290730e-01
     -6.290730e-01,-6.290730e-01 , 6.290730e-01
      3.629073e+00, 6.290730e-01 , 6.290730e-01
      3.629073e+00,-6.290730e-01 ,-6.290730e-01
      2.370927e+00, 6.290730e-01 ,-6.290730e-01
      2.370927e+00,-6.290730e-01 , 6.290730e-01
      6.290730e-01, 3.629073e+00 , 6.290730e-01
     6.290730e-01,  2.370927e+00 ,-6.290730e-01
     -6.290730e-01,  3.629073e+00 ,-6.290730e-01
     -6.290730e-01,  2.370927e+00 , 6.290730e-01
     6.290730e-01,  6.290730e-01 , 3.629073e+00
     6.290730e-01, -6.290730e-01 , 2.370927e+00
     -6.290730e-01,  6.290730e-01,  2.370927e+00
     -6.290730e-01, -6.290730e-01,  3.629073e+00"""
    coords = ''
    for line in coord.split('\n'):
        x,y,z = map(float,line.replace(' ','').split(','))
        coords += f' H {rotation_matrix_from_euler(roll, pitch, yaw, [x,y,z])} ; '
    mol.build( atom = coords, basis ='STO-3G',spin= 0)
    filename = 'H16'
Run['filename'] = filename

In [74]:
# Select state 
ne = mol.nelectron//2
canons = 4
iterate = 100
total_iterate = 100
alpha=1e-4
Run['ne']     = ne 
Run['iterate'] = iterate
Run['total_iterate'] = total_iterate
Run['canons']=canons
Run['alpha'] = alpha
ne


8

In [75]:
## RHF ##
##     ##
mf = scf.RHF(mol).run()
orb = mf.mo_coeff ##First Level##
if len(np.shape(orb))< 3:
    orb = [orb]
np.shape(orb)

SCF not converged.
SCF energy = -6.24798034741421


(1, 16, 16)

In [ ]:
Run['box_size'] = box_size
Run['L']        = L
box_size = 10.0
L        = 500

lattices = 3*[ np.linspace( -box_size/2,box_size/2, L ) ]

Sum1 = get_orbital(mol, orb[0], ne-1, lattices=lattices)
Sum1 = Sum1.mul(1./Sum1.n())

In [77]:
def D(q, method, Sum1=Sum1):
    q1 = q.copy().mul(1./q.n())
    return pd.Series({'method':method, 'dist':(q1.dist(Sum1)).numpy(), 'canons':int(len(q1)), 'ld2':q1.ld2().numpy(), 'ld1':q1.ld1().numpy(),'norm':q1.n().numpy()})

The output of get_orbital are in digital space properly, they are orthonormal

In [78]:
Sum1.dot(Sum1),len(Sum1)

(<tf.Tensor: shape=(), dtype=float64, numpy=1.0000000000000004>, 48)

In [79]:
q = Sum1.copy().fibonacci(  partition=canons, iterate=iterate, total_iterate=total_iterate, alpha=alpha)

D(q,'fibonacci')

method    fibonacci
dist       0.490773
canons            4
ld2         1.04549
ld1        1.946546
norm            1.0
dtype: object

In [80]:
q2 = Sum1.copy().boost().fibonacci(  partition=canons, iterate=iterate, total_iterate=total_iterate, alpha=alpha).unboost()

D(q2,'boost_fibonacci')

method    boost_fibonacci
dist             0.526197
canons                  4
ld2              0.937266
ld1              1.790719
norm                  1.0
dtype: object

In [81]:
q3 = Sum1.copy().boost().decompose(  partition=canons, iterate=iterate, alpha=alpha).unboost()

D(q3,'boost_decompose')

method    boost_decompose
dist             0.618058
canons                  4
ld2              1.001544
ld1              1.728892
norm                  1.0
dtype: object

In [82]:
q4 = Sum1.copy().decompose(  partition=canons, iterate=iterate, alpha=alpha)

D(q4,'decompose')

method    decompose
dist       0.610132
canons            4
ld2        1.012203
ld1        1.772769
norm            1.0
dtype: object

In [83]:
Tum1 = { 0: Sum1[0], 1: Sum1[1], 2: Sum1[2], 3: Sum1[3] } 
lattices_dict = { 1:lattices[0], 2:lattices[1], 3:lattices[2]}

In [104]:
tl = ext.reduce(ext.image(Tum1), canons )

tl1 = sp.vector().transpose(tl, lattices_dict).copy(norm_ = True)
tl1[0]


<tf.Tensor: shape=(4, 1), dtype=float64, numpy=
array([[1.00700782],
       [0.59923237],
       [0.99884253],
       [0.57258941]])>

In [107]:
D(tl1,'reduce')
#tl1[0]

method      reduce
dist       0.14089
canons           4
ld2       1.659237
ld1       3.209527
norm           1.0
dtype: object

In [103]:
pd.DataFrame(([D(Sum1,'Orbital'),D(q,'fibonacci'),D(q2,'boost_fibonacci'),D(q3,'boost_decompose'), D(q4, 'decompose'),D(tl1,'reduce')]))

,method,dist,canons,ld2,ld1,norm
0,Orbital,2.580957e-08,48,1.101481,5.959204,1.0
1,fibonacci,4.907729e-01,4,1.045490,1.946546,1.0
2,boost_fibonacci,5.261965e-01,4,0.937266,1.790719,1.0
3,boost_decompose,6.180581e-01,4,1.001544,1.728892,1.0
4,decompose,6.101325e-01,4,1.012203,1.772769,1.0
5,reduce,1.477100e-01,4,1.613261,3.180537,1.0
